# 클래스

In [57]:
!pip install ipycanvas >/dev/null

In [63]:
%load_ext google.colab.data_table

In [132]:
from ipycanvas import Canvas, hold_canvas
import pandas as pd
import numpy as np
import pickle
from uuid import UUID, uuid4
from dataclasses import dataclass, field
from IPython.display import display
from typing import Optional, List

def factory(factory):
  return field(default_factory=factory)

In [60]:
@dataclass
class Car:
  color: str = '#ff0000'
  wheel_size: float = 16
  displacement: float = 2000
  

  def forward(self):
    self.displacement += 100
  
  def backward(self):
    self.displacement -= 100
  
  def turn_left(self):
    pass
  
  def turn_right(self):
    pass
  
  def widget(self):
    canvas = Canvas(width = 400, height = 300)
    with hold_canvas(canvas):
      canvas.fill_style = self.color
      canvas.fill_rect(0, 0, canvas.width, canvas.height)
    return canvas

  def __repr__(self): # when coercing an object into string. print(car) will print the result of tihs
    return f'자동차 색: {self.color}, 바퀴 크기: {self.wheel_size}, 변위: {self.displacement}'

In [59]:
car1 = Car(color = '#ff3300')
car1.forward()

In [68]:
canvas = Canvas(width = 400, height = 3000)
canvas.fill_style = 'black'
canvas.fill_rect(0, 0, canvas.width, canvas.height)
canvas

Canvas(height=3000, width=4000)

In [81]:
class ClassVar:
  # instance property
  # I think this is not a good practice...
  text_list: List[str] = [] # Default value is shared as class attribute
  global_var = 1

  def add(self, text):
    self.text_list.append(text)
    self.global_var += 1
  
  def print_list(self):
    print(self.text_list)
    print(self.global_var)

In [82]:
a = ClassVar()
a.add('a')
a.print_list()

b = ClassVar()
b.add('b')

a.print_list()
b.print_list()

['a']
2
['a', 'b']
2
['a', 'b']
2


In [85]:
@dataclass
class ClassVar2:
  # instance property
  text_list: List[str] = factory(lambda: [])

  def add(self, text):
    self.text_list.append(text)
  
  def print_list(self):
    print(self.text_list)

In [86]:
a = ClassVar2()
a.add('a')
a.print_list()

b = ClassVar2()
b.add('b')

a.print_list()
b.print_list()

['a']
['a']
['b']


## 주소록 클래스 설계

In [150]:
# 주소록 클래스 설계

@dataclass
class Name:
  first_name: str
  last_name: str
  reference_subject: Optional[str] = None # for example, he/she
  reference_object: Optional[str] = None # for example, him/her


  def __repr__(self):
    return f'{self.first_name} {self.last_name}'


@dataclass
class User:
  name: Name
  uuid: UUID = factory(lambda: uuid4())
  phone_number: Optional[str] = None
  email: Optional[str] = None
  metadata: dict = factory(lambda: {})


  def table(self):
    return [str(self.uuid), self.name, self.phone_number, self.email]

  def __repr__(self):
    text = f'{self.name}'
    if self.phone_number != None: text += f', phone number: {self.phone_number}'
    if self.email != None: text += f', email: {self.email}'
    
    return text


@dataclass
class Contact:
  all_users: List[User]


  def table(self):
    return pd.DataFrame([user.table() for user in self.all_users], columns=['UUID', 'name', 'phone number', 'email'])

  def __repr__(self):
    return '\n'.join([repr(item) for item in self.all_users])

In [153]:
contact = Contact(
  all_users = [
    User(name='이현우', phone_number='010-1234-5678'),
    User(name='어머니', phone_number='010-1111-2222'),
    User(name='아버지', phone_number='010-3333-4444', email='hello_world@example.com'),
    User(name='할아버지', phone_number='010-5555-6666')
  ]
)

contact.table()

,UUID,name,phone number,email
0,483b9305-592a-4f1a-8b19-f3d5a6f27539,이현우,010-1234-5678,None
1,187a3428-a259-44ce-8e7e-4d48dddb26d8,어머니,010-1111-2222,None
2,cc0c5b87-00b5-4c13-bd98-090612f18bc8,아버지,010-3333-4444,hello_world@example.com
3,ba7eeae1-51b1-4f15-9b87-9bc361859912,할아버지,010-5555-6666,None


In [154]:
file_name = 'contact.txt'

with open(file_name, 'wb') as file:
  pickle.dump(contact, file)

In [155]:
with open(file_name, 'rb') as file:
  contact2 = pickle.load(file) # !!vulerability warning!! this is stub implementation, should be changed in real world implementations

print(contact2)

이현우, phone number: 010-1234-5678
어머니, phone number: 010-1111-2222
아버지, phone number: 010-3333-4444, email: hello_world@example.com
할아버지, phone number: 010-5555-6666
